## Initial analysis on the data to see what I have to work with and what data I would like to get

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pickle


In [2]:
file_path = '../data/all_data.csv'
data = pd.read_csv(file_path)
data.head()

,artists,duration_ms_x,explicit,id_x,name,popularity,search_query,acousticness,danceability,duration_ms_y,...,mode_confidence,num_samples,offset_seconds,sample_md5,start_of_fade_out,tempo_y,tempo_confidence,time_signature_y,time_signature_confidence,window_seconds
0,"2 Chainz, Travis Scott",255560.0,True,1nX9KhK3Fff27SnrIor2Yb,4 AM,72.0,2 chainz 4 am,0.1180,0.796,255560.0,...,0.516,5635098.0,0.0,NaN,252.70857,75.012,0.580,4.0,1.000,0.0
1,"2 Chainz, Ty Dolla $ign, Trey Songz, Jhene Aiko",210200.0,True,6H0AwSQ20mo62jGlPGB8S6,It's A Vibe,76.0,2 chainz it's a vibe,0.0317,0.822,210200.0,...,0.436,4634910.0,0.0,NaN,198.87891,73.003,0.297,4.0,1.000,0.0
2,"2 Chainz, YG, Offset",234666.0,True,365wwIjijQdlRJEjUWTidq,PROUD,61.0,2 chainz proud,0.7380,0.781,234667.0,...,0.499,5174400.0,0.0,NaN,230.01687,173.949,0.578,4.0,1.000,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,2 chainz x gucci mane x quavo good drank,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"2 Chainz, Drake, Quavo",225893.0,True,5S1IUPueD0xE0vj4zU3nSf,Bigger Than You (feat. Drake & Quavo),75.0,"2 chainz, drake bigger > you",0.2470,0.888,225893.0,...,0.700,4980948.0,0.0,NaN,221.88408,141.009,0.753,4.0,0.971,0.0


## Need to do some EDA
1. What is the distribution of songs
2. who are the most popular and who are the least popular artists
3. what currently is the most correlated with the number of weeks a song is in the billboards
    - hey have i considered the time that the song has been released? because it doesnt give new songs that chance to have the same amount of time as older songs!!!!!

In [3]:
data.columns

Index(['artists', 'duration_ms_x', 'explicit', 'id_x', 'name', 'popularity',
       'search_query', 'acousticness', 'danceability', 'duration_ms_y',
       'energy', 'id_y', 'instrumentalness', 'key_x', 'liveness', 'loudness_x',
       'mode_x', 'speechiness', 'tempo_x', 'time_signature_x', 'uri',
       'valence', 'analysis_channels', 'analysis_sample_rate', 'duration',
       'end_of_fade_in', 'id', 'key_y', 'key_confidence', 'loudness_y',
       'mode_y', 'mode_confidence', 'num_samples', 'offset_seconds',
       'sample_md5', 'start_of_fade_out', 'tempo_y', 'tempo_confidence',
       'time_signature_y', 'time_signature_confidence', 'window_seconds'],
      dtype='object')

In [4]:
# seems like I have some repeat columns so I'm going to drop those

drop_repeats = [column for column in data.columns if '_y' in column]
# there is an extra id column so adding that to the list
drop_repeats.append('id')
drop_repeats
data.drop(drop_repeats, axis=1, inplace=True)
data.columns

Index(['artists', 'duration_ms_x', 'explicit', 'id_x', 'name', 'popularity',
       'search_query', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key_x', 'liveness', 'loudness_x', 'mode_x',
       'speechiness', 'tempo_x', 'time_signature_x', 'uri', 'valence',
       'analysis_channels', 'analysis_sample_rate', 'duration',
       'end_of_fade_in', 'key_confidence', 'mode_confidence', 'num_samples',
       'offset_seconds', 'sample_md5', 'start_of_fade_out', 'tempo_confidence',
       'time_signature_confidence', 'window_seconds'],
      dtype='object')

In [5]:
rename_columns = [column.replace('_x', '') for column in data.columns]

data.columns = rename_columns
# checking my work
data.columns

Index(['artists', 'duration_ms', 'explicit', 'id', 'name', 'popularity',
       'search_query', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'uri', 'valence',
       'analysis_channels', 'analysis_sample_rate', 'duration',
       'end_of_fade_in', 'key_confidence', 'mode_confidence', 'num_samples',
       'offset_seconds', 'sample_md5', 'start_of_fade_out', 'tempo_confidence',
       'time_signature_confidence', 'window_seconds'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 33 columns):
artists                      809 non-null object
duration_ms                  809 non-null float64
explicit                     809 non-null object
id                           809 non-null object
name                         809 non-null object
popularity                   809 non-null float64
search_query                 828 non-null object
acousticness                 809 non-null float64
danceability                 809 non-null float64
energy                       809 non-null float64
instrumentalness             809 non-null float64
key                          809 non-null float64
liveness                     809 non-null float64
loudness                     809 non-null float64
mode                         809 non-null float64
speechiness                  809 non-null float64
tempo                        809 non-null float64
time_signature               809 non-null float64


In [7]:
# sample_md5 is missing values so I should drop that 

data.drop('sample_md5', axis=1, inplace=True)

In [8]:
data.head()

,artists,duration_ms,explicit,id,name,popularity,search_query,acousticness,danceability,energy,...,duration,end_of_fade_in,key_confidence,mode_confidence,num_samples,offset_seconds,start_of_fade_out,tempo_confidence,time_signature_confidence,window_seconds
0,"2 Chainz, Travis Scott",255560.0,True,1nX9KhK3Fff27SnrIor2Yb,4 AM,72.0,2 chainz 4 am,0.1180,0.796,0.500,...,255.56000,0.00000,0.271,0.516,5635098.0,0.0,252.70857,0.580,1.000,0.0
1,"2 Chainz, Ty Dolla $ign, Trey Songz, Jhene Aiko",210200.0,True,6H0AwSQ20mo62jGlPGB8S6,It's A Vibe,76.0,2 chainz it's a vibe,0.0317,0.822,0.505,...,210.20000,0.00000,0.281,0.436,4634910.0,0.0,198.87891,0.297,1.000,0.0
2,"2 Chainz, YG, Offset",234666.0,True,365wwIjijQdlRJEjUWTidq,PROUD,61.0,2 chainz proud,0.7380,0.781,0.810,...,234.66667,0.08689,0.353,0.499,5174400.0,0.0,230.01687,0.578,1.000,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,2 chainz x gucci mane x quavo good drank,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"2 Chainz, Drake, Quavo",225893.0,True,5S1IUPueD0xE0vj4zU3nSf,Bigger Than You (feat. Drake & Quavo),75.0,"2 chainz, drake bigger > you",0.2470,0.888,0.515,...,225.89333,2.51356,0.802,0.700,4980948.0,0.0,221.88408,0.753,0.971,0.0


In [10]:
# i want to know what songs are missing so that i can manually get those since i want to get as much data as possible

data[data.explicit.isna()].to_csv('../data/missing_data_songs.csv', index=False)

## Cleaning Data:

Going through each of the columns, what is the best way to handle it

#### artists columns

In [ ]:
data['artists'].fillna('missing', inplace=True)
artists_df = data['artists'].apply(lambda x: x.split(', '))

In [ ]:
artists_df = pd.get_dummies(artists_df.apply(pd.Series).stack()).sum(level=0)

In [ ]:
artists_df.sum().sort_values(ascending=False)

NTS: what does the evolution of Drake sound like (according to the data?)

#### explict
Python already knows that  true to 1 and false to 0

In [ ]:
data.columns

In [ ]:
set(data['analysis_channels'].values) # that means i need to drop this column since it provides no signal

In [ ]:
set(data['analysis_sample_rate'].values) # that means i need to drop this column since it provides no signal

In [ ]:
data['end_of_fade_in'].hist(); # what does this even mean though


In [ ]:
data['key_confidence'].hist() # what does this even mean

# i would like know the songs that have low confidence what do they have in common and i want to know
# why they have low confidence like whats wrong bb it ok!!

In [ ]:
data['num_samples'].hist() # what does this mean

In [ ]:
pickle.dump(artists_df, open('data/artists_df_dummied.pkl', 'wb'))